In [2]:
# 初始化环境
import os

os.environ['OPENAI_BASE_URL'] = 'https://key.wenwen-ai.com/v1'
os.environ['OPENAI_API_KEY'] = 'sk-6V2exWFBSa2lmuZ7C0D773D1BaEd4fB7A1B6A0A265D550C6'

In [12]:
# 一般参数
from openai import OpenAI
from openai.types.chat import completion_create_params

client = OpenAI()

completion = client.chat.completions.create(
    model="gpt-3.5-turbo-1106",
    messages=[
        {"role": "system", "content": "You are a helpful assistant."},
        {"role": "user", "content": "Hello! "}
    ],
    
    messages1=[
        {"role": "system", "content": "You are a helpful assistant."},
        {"role": "user", "content": f"Hello!{client} "}
    ],
    n=1,
    max_tokens=2048,
    temperature=0.8,
    # top_p=1,
    # stop="!",
    seed=1234567,
    # user="user_1234567",
)

print(completion.model_dump_json())

{"id": "chatcmpl-8hajhipm8fwxXrFMDUvjXEbAY30lJ", "choices": [{"finish_reason": "stop", "index": 0, "message": {"content": "Hello! How can I assist you today?", "role": "assistant", "function_call": null, "tool_calls": null}, "logprobs": null}], "created": 1705400009, "model": "gpt-3.5-turbo-1106", "object": "chat.completion", "system_fingerprint": "fp_cbe4fa03fe", "usage": {"completion_tokens": 9, "prompt_tokens": 21, "total_tokens": 30}}


In [3]:
# response_format 参数 https://platform.openai.com/docs/guides/text-generation/json-mode
from openai import OpenAI
from openai.types.chat import completion_create_params

client = OpenAI()

completion = client.chat.completions.create(
    model="gpt-3.5-turbo-1106",
    messages=[
        {"role": "system", "content": "You are a helpful assistant designed to output JSON."},
        {"role": "user", "content": "Who won the world series in 2020?"}
    ],
    # 对content进行json格式化
    response_format=completion_create_params.ResponseFormat(type="json_object")
)

print(completion.choices[0].message.model_dump_json())

{"content":"{\n  \"winner\": \"Los Angeles Dodgers\"\n}","role":"assistant","function_call":null,"tool_calls":null}


In [1]:
# stream流式输出
from openai import OpenAI

client = OpenAI()

completion = client.chat.completions.create(
    model="gpt-3.5-turbo-1106",
    messages=[
        {"role": "system", "content": "You are a helpful assistant."},
        {"role": "user", "content": "Hello!"}
    ],
    stream=True
)

for chunk in completion:
    print(chunk.choices[0].delta.model_dump_json())


{"content":"","function_call":null,"role":"assistant","tool_calls":null}
{"content":"Hello","function_call":null,"role":null,"tool_calls":null}
{"content":"!","function_call":null,"role":null,"tool_calls":null}
{"content":" How","function_call":null,"role":null,"tool_calls":null}
{"content":" can","function_call":null,"role":null,"tool_calls":null}
{"content":" I","function_call":null,"role":null,"tool_calls":null}
{"content":" assist","function_call":null,"role":null,"tool_calls":null}
{"content":" you","function_call":null,"role":null,"tool_calls":null}
{"content":" today","function_call":null,"role":null,"tool_calls":null}
{"content":"?","function_call":null,"role":null,"tool_calls":null}
{"content":null,"function_call":null,"role":null,"tool_calls":null}


In [27]:
# logit_bias参数
from openai import OpenAI
import tiktoken

enc = tiktoken.get_encoding("cl100k_base")
print(enc.encode("Hello"))

client = OpenAI()

completion = client.chat.completions.create(
    model="gpt-3.5-turbo-1106",
    messages=[
        {"role": "system", "content": "You are a helpful assistant."},
        {"role": "user", "content": "Hello!"}
    ],
    logit_bias={enc.encode("Hello")[0]: -100}
)

print(completion.choices[0].message.model_dump_json())

[9906]
{"content":"Hi there! How can I assist you today?","role":"assistant","function_call":null,"tool_calls":null}


In [5]:
# logprobs参数
from openai import OpenAI

client = OpenAI()

completion = client.chat.completions.create(
    model="gpt-3.5-turbo-1106",
    messages=[
        {"role": "user", "content": "Hello!"}
    ],
    logprobs=True,
    top_logprobs=2
)

print(completion.choices[0].message.model_dump_json())
print(completion.choices[0].logprobs.model_dump_json())

{"content":"Hello! How can I assist you today?","role":"assistant","function_call":null,"tool_calls":null}
{"content":[{"token":"Hello","bytes":[72,101,108,108,111],"logprob":-0.21570958,"top_logprobs":[{"token":"Hello","bytes":[72,101,108,108,111],"logprob":-0.21570958},{"token":"Hi","bytes":[72,105],"logprob":-1.6720008}]},{"token":"!","bytes":[33],"logprob":-0.20926832,"top_logprobs":[{"token":"!","bytes":[33],"logprob":-0.20926832},{"token":" there","bytes":[32,116,104,101,114,101],"logprob":-1.6910303}]},{"token":" How","bytes":[32,72,111,119],"logprob":-0.00095136015,"top_logprobs":[{"token":" How","bytes":[32,72,111,119],"logprob":-0.00095136015},{"token":" Is","bytes":[32,73,115],"logprob":-7.526584}]},{"token":" can","bytes":[32,99,97,110],"logprob":-0.014271167,"top_logprobs":[{"token":" can","bytes":[32,99,97,110],"logprob":-0.014271167},{"token":" are","bytes":[32,97,114,101],"logprob":-4.673554}]},{"token":" I","bytes":[32,73],"logprob":-0.00001700133,"top_logprobs":[{"tok

In [17]:
from openai import OpenAI

client = OpenAI()

tools = [
    {
        "type": "function",
        "function": {
            "name": "get_current_weather",
            "description": "Get the current weather in a given location",
            "parameters": {
                "type": "object",
                "properties": {
                    "location": {
                        "type": "string",
                        "description": "The city and state, e.g. San Francisco, CA",
                    },
                    "unit": {"type": "string", "enum": ["celsius", "fahrenheit"]},
                },
                "required": ["location"],
            },
        }
    }
]
messages = [{"role": "user", "content": "What's the weather like in Boston today?"}]
completion = client.chat.completions.create(
    model="gpt-3.5-turbo-1106",
    messages=messages,
    tools=tools,
    tool_choice="auto"
)

print(completion.model_dump_json())

{"id":"chatcmpl-8a338gVY0y65AzAMrajrFvVHrXPVV","choices":[{"finish_reason":"tool_calls","index":0,"logprobs":null,"message":{"content":null,"role":"assistant","function_call":null,"tool_calls":[{"id":"call_yAjy5Ktcqn1unsqt9XgPRWku","function":{"arguments":"{\"location\":\"Boston, MA\",\"unit\":\"celsius\"}","name":"get_current_weather"},"type":"function"}]}}],"created":1703602222,"model":"gpt-3.5-turbo-1106","object":"chat.completion","system_fingerprint":"fp_772e8125bb","usage":{"completion_tokens":22,"prompt_tokens":83,"total_tokens":105}}


In [1]:
from openai import OpenAI

client = OpenAI()

response = client.chat.completions.create(
    model="gpt-4-vision-preview",
    messages=[
        {
            "role": "user",
            "content": [
                {"type": "text", "text": "What’s in this image?"},
                {
                    "type": "image_url",
                    "image_url": "https://upload.wikimedia.org/wikipedia/commons/thumb/d/dd/Gfp-wisconsin-madison-the-nature-boardwalk.jpg/2560px-Gfp-wisconsin-madison-the-nature-boardwalk.jpg",
                },
            ],
        }
    ],
    max_tokens=300,
)

print(response.choices[0].model_dump_json())


{"finish_reason":"stop","index":0,"logprobs":null,"message":{"content":"The image shows a beautiful natural landscape with a wooden boardwalk extending straight into the distance. The boardwalk is surrounded by lush green grasses and foliage. The sky is bright and partly cloudy, suggesting a sunny day with good weather. In the distance, you can see a line of trees and some shrubbery that demarcates the end of the open field. This looks like a peaceful setting for a walk or to enjoy nature.","role":"assistant","function_call":null,"tool_calls":null}}
